## Libraries

In [16]:
#
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# utils
# from scipy.misc import imread
# from scipy.misc import imresize
import os
import random

# Configure visualisations
%matplotlib inline
color = sns.color_palette()
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 10,8 


In [17]:
# set the root path

import os
import pandas as pd
root_dir = os.path.abspath('.')
# set the data dir
data_dir = '../input'

## Import train and test csv files

In [18]:
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [19]:
train.head()

,File_name,Age
0,100_1_0_20170110183726390.jpg.chip.jpg,100
1,100_1_2_20170105174847679.jpg.chip.jpg,100
2,101_1_2_20170105174739309.jpg.chip.jpg,101
3,10_0_0_20161220222308131.jpg.chip.jpg,10
4,10_0_0_20170103200329407.jpg.chip.jpg,10


In [20]:
test.head()

,File_name,Age
0,10_0_0_20170103200522151.jpg.chip.jpg,10
1,10_0_0_20170110220447314.jpg.chip.jpg,10
2,10_0_0_20170110221714752.jpg.chip.jpg,10
3,10_0_0_20170110224524253.jpg.chip.jpg,10
4,10_0_0_20170110224725285.jpg.chip - Copy.jpg,10


In [21]:
print("Train shape :",train.shape)
print("Test shape :",test.shape)


Train shape : (8255, 2)
Test shape : (1635, 2)


In [22]:
def show_random_image(df, folder_name):
    i = random.choice(df.index)

    img_name = df.File_name[i]
    img = imread(os.path.join(data_dir, folder_name, img_name))

    print('Age : ', train.Age[i])
    imshow(img)
    

In [23]:
# show random image 
import random
from cv2 import imread,imshow

show_random_image(train, "train")

Age :  39


TypeError: imshow() missing required argument 'mat' (pos 2)

## Resize

In [24]:
import cv2
def resize_img(df, folder_name):
    temp = []
    for index,img_name in enumerate(df.File_name):
        if index % 1000 == 0:
            print( 'Index :',index)
        img_path = os.path.join(data_dir, folder_name, img_name)
        img = imread(img_path)
        img = cv2.resize(img, (32, 32))
        img = img.astype('float32')
        temp.append(img)

    return np.stack(temp)
    

In [25]:
%%time
# resize all the images 32 x 32 (training set), usually takes ~ 4-5 min (CPU)
train_x = resize_img(train, "train")

Index : 0
Index : 1000
Index : 2000
Index : 3000
Index : 4000
Index : 5000
Index : 6000
Index : 7000
Index : 8000
Wall time: 8.25 s


In [26]:
# resize all the images 32 x 32 (testing set)
test_x = resize_img(test, "test")

Index : 0
Index : 1000


In [33]:
# flatten the image array
train_x = train_x / 255
test_x = test_x / 255
train_x

array([[[[0.95686275, 0.91764706, 0.8862745 ],
         [0.57254905, 0.4627451 , 0.39607844],
         [0.46666667, 0.36862746, 0.30980393],
         ...,
         [0.14901961, 0.1254902 , 0.10980392],
         [0.12941177, 0.09019608, 0.09019608],
         [0.9137255 , 0.88235295, 0.85490197]],

        [[0.7058824 , 0.6392157 , 0.6       ],
         [0.63529414, 0.50980395, 0.45490196],
         [0.45882353, 0.3882353 , 0.3647059 ],
         ...,
         [0.07450981, 0.0627451 , 0.07450981],
         [0.17254902, 0.12941177, 0.13725491],
         [0.8235294 , 0.7921569 , 0.7647059 ]],

        [[0.4862745 , 0.38431373, 0.33333334],
         [0.4862745 , 0.34901962, 0.3019608 ],
         [0.20784314, 0.16862746, 0.16470589],
         ...,
         [0.02352941, 0.03921569, 0.07058824],
         [0.1764706 , 0.13725491, 0.13725491],
         [0.74509805, 0.7176471 , 0.6862745 ]],

        ...,

        [[0.7137255 , 0.4117647 , 0.16078432],
         [0.6666667 , 0.4392157 , 0.24313726]

In [30]:
from keras import utils
from keras.utils.np_utils import to_categorical

In [35]:
import keras

train_y = train.Age
train_y = to_categorical(train_y)
train_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [36]:
# set up the parameters for neural network
input_num_units = (32, 32, 3)
hidden_num_units = 600
output_num_units = 3
num_filter = 100

epochs = 30
batch_size = 64

## MOdeling

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D, Dropout

In [38]:
model = Sequential()

## Input Layer
model.add(InputLayer(input_shape=input_num_units))

## Convolutional layer
model.add(Conv2D(num_filter, (3, 3), activation='relu', input_shape=input_num_units))

model.add(MaxPooling2D())

model.add(Dropout(0.25))

model.add(Conv2D(int(num_filter * 0.5), (3, 3), activation='relu', input_shape=(10, 10, 100)))

model.add(MaxPooling2D())

model.add(Dropout(0.25))

model.add(Conv2D(int(num_filter * 0.3), (3, 3), activation='relu', input_shape=(1, 1, 10)))

model.add(Flatten())

model.add(Dense(hidden_num_units, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(int(hidden_num_units * 0.2), activation='relu'))

model.add(Dense(1, kernel_initializer='normal',activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 100)       2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 100)       0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 50)        45050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 50)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 50)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 30)          1

In [39]:
from keras.callbacks import ModelCheckpoint

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [40]:
model.fit(train_x, train_y, batch_size=batch_size,epochs=100,verbose=1, validation_split=0.1,callbacks=callbacks_list)

Epoch 1/100
116/117 [============================>.] - ETA: 0s - loss: 0.0103 - mean_absolute_error: 0.0103
Epoch 00001: val_loss improved from inf to 0.00919, saving model to Weights-001--0.00919.hdf5
117/117 [==============================] - 13s 110ms/step - loss: 0.0103 - mean_absolute_error: 0.0103 - val_loss: 0.0092 - val_mean_absolute_error: 0.0092
Epoch 2/100
116/117 [============================>.] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.0091
Epoch 00002: val_loss improved from 0.00919 to 0.00906, saving model to Weights-002--0.00906.hdf5
117/117 [==============================] - 12s 106ms/step - loss: 0.0091 - mean_absolute_error: 0.0091 - val_loss: 0.0091 - val_mean_absolute_error: 0.0091
Epoch 3/100
116/117 [============================>.] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.0091
Epoch 00003: val_loss did not improve from 0.00906
117/117 [==============================] - 14s 118ms/step - loss: 0.0091 - mean_absolute_error: 0.0091 - val_loss: 0.0091

Epoch 52/100
116/117 [============================>.] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.0091
Epoch 00052: val_loss did not improve from 0.00901
117/117 [==============================] - 16s 141ms/step - loss: 0.0091 - mean_absolute_error: 0.0091 - val_loss: 0.0092 - val_mean_absolute_error: 0.0092
Epoch 53/100
116/117 [============================>.] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.0091
Epoch 00053: val_loss did not improve from 0.00901
117/117 [==============================] - 16s 137ms/step - loss: 0.0091 - mean_absolute_error: 0.0091 - val_loss: 0.0093 - val_mean_absolute_error: 0.0093
Epoch 54/100
116/117 [============================>.] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.0091
Epoch 00054: val_loss did not improve from 0.00901
117/117 [==============================] - 17s 149ms/step - loss: 0.0091 - mean_absolute_error: 0.0091 - val_loss: 0.0093 - val_mean_absolute_error: 0.0093
Epoch 55/100
116/117 [============================>.

Epoch 78/100
116/117 [============================>.] - ETA: 0s - loss: 0.0092 - mean_absolute_error: 0.0092
Epoch 00078: val_loss did not improve from 0.00901
117/117 [==============================] - 20s 171ms/step - loss: 0.0092 - mean_absolute_error: 0.0092 - val_loss: 0.0091 - val_mean_absolute_error: 0.0091
Epoch 79/100
116/117 [============================>.] - ETA: 0s - loss: 0.0092 - mean_absolute_error: 0.0092
Epoch 00079: val_loss did not improve from 0.00901
117/117 [==============================] - 18s 150ms/step - loss: 0.0092 - mean_absolute_error: 0.0092 - val_loss: 0.0093 - val_mean_absolute_error: 0.0093
Epoch 80/100
116/117 [============================>.] - ETA: 0s - loss: 0.0092 - mean_absolute_error: 0.0092
Epoch 00080: val_loss did not improve from 0.00901
117/117 [==============================] - 17s 143ms/step - loss: 0.0092 - mean_absolute_error: 0.0092 - val_loss: 0.0091 - val_mean_absolute_error: 0.0091
Epoch 81/100
116/117 [============================>.

## Load best model

In [45]:
weights_file = 'Weights-020--0.00901.hdf5' # choose the best checkpoint 
model.load_weights(weights_file) # load it
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [52]:
pred_train = model.predict_classes(train_x, verbose=1)
pred = model.predict_classes(test_x, verbose=1)


test['Age'] = pred
test.to_csv('submission.csv', index=False)

52/52 [==============================] - 0s 9ms/step


In [53]:
submission = pd.read_csv('submission.csv')
submission

,File_name,Age,Class
0,10_0_0_20170103200522151.jpg.chip.jpg,0,0
1,10_0_0_20170110220447314.jpg.chip.jpg,0,0
2,10_0_0_20170110221714752.jpg.chip.jpg,0,0
3,10_0_0_20170110224524253.jpg.chip.jpg,0,0
4,10_0_0_20170110224725285.jpg.chip - Copy.jpg,0,0
...,...,...,...
1630,9_0_0_20170110225419379.jpg.chip.jpg,0,0
1631,9_0_1_20170110224349623.jpg.chip.jpg,0,0
1632,9_0_2_20170110225353988.jpg.chip.jpg,0,0
1633,9_1_0_20170109203259973.jpg.chip.jpg,0,0


## Predictions

In [58]:
i = random.choice(train.index)
img_name = train.File_name[i]

img = cv2.imread(os.path.join(data_dir, 'train', img_name)).astype('float32')
#imshow(cv2.resize(img, (64, 64)))
print('Predicted:',pred_train[i])

Predicted: [0]


In [64]:
def test_image(ind,images_f,images_f_2,Model):
  cv2_imshow(images_f[ind])
  image_test=images_f_2[ind]
  pred_1=Model.predict(np.array([image_test]))
  #print(pred_1)
 # sex_f=['Male','Female']
  age=int(np.round(pred_1[1][0]))
  #sex=int(np.round(pred_1[0][0]))
  print("Predicted Age: "+ str(age))
  #print("Predicted Sex: "+ sex_f[sex])



In [65]:
test_image('df',images_f,images_f_2,Model)

NameError: name 'images_f' is not defined

In [59]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
#model1=load_model('model_vgg16.h5')
model
d={}
dirr = r'C:\Users\piyush.pathak\Desktop\age prediction\input\new_data'
os.chdir(dirr)
all_images = os.listdir()
print(all_images)
for i in range(len(all_images)):
    img=image.load_img(all_images[i],target_size=(224,224))
    x=image.img_to_array(img)

    x=x/255
    x=np.expand_dims(x,axis=0)
    model.predict(x)
    d[all_images[i]]=np.argmax(model.predict(x), axis=1)
for key,values in d.items():
    print(str(key)+"/"+str(values))

['12324325.jpg', 'df.jpg', 'fgfgbrgf.jpg']


ValueError: in user code:

    C:\Users\piyush.pathak\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\piyush.pathak\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\piyush.pathak\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\piyush.pathak\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\piyush.pathak\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\piyush.pathak\Anaconda3\lib\site-packages\tensorflo